In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import sys
sys.path.append('/home/mahdi_mousavi/comb-auction')

In [3]:
import torch
import torch.nn as nn
import logging

from comblearn.env import CombinatorialAuction

logging.basicConfig(level=20)

In [4]:
import yaml 

with open("/home/mahdi_mousavi/comb-auction/notebooks/config_submod_mini.yaml") as fp:
        cfg = yaml.load(fp, Loader=yaml.FullLoader)

In [5]:
device = "cuda"

def social_welfare(ws, allocation):
    return torch.sum(torch.tensor([w(alloc) for w, alloc in zip(ws, allocation)]).to(device))

In [6]:
auction = CombinatorialAuction(cfg['auction'])

social welfare is: 73.0, iteration is: 1000
social welfare is: 96.0, iteration is: 2000
social welfare is: 97.0, iteration is: 3000
social welfare is: 87.0, iteration is: 4000
social welfare is: 110.0, iteration is: 5000
social welfare is: 89.0, iteration is: 6000
[tensor([0., 1., 0., 0., 0., 0., 1., 1.], device='cuda:0'), tensor([0., 0., 0., 1., 1., 1., 0., 0.], device='cuda:0'), tensor([1., 0., 1., 0., 0., 0., 0., 0.], device='cuda:0')]
Brute Force Search: done.
Optimal Social Welfare: 117.0


In [7]:
#allocations, payments = auction.run()
allocation, social_welfare = auction.run()

INFO:root:Final allocation calculation...
/usr/lib/python3/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([500, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
INFO:root:Bidder ali, loss: 0.0
INFO:root:Bidder hassan, loss: 1.5079507331402023e-14
INFO:root:Bidder mahdi, loss: 0.0
INFO:root:t: 0.2500000000000001/1
INFO:root:t: 0.5000000000000003/1
INFO:root:t: 0.7500000000000006/1
INFO:root:t: 1.0000000000000007/1
INFO:root:t: 0.2500000000000001/1
INFO:root:t: 0.5000000000000003/1
INFO:root:t: 0.7500000000000006/1
INFO:root:t: 1.0000000000000007/1
INFO:root:Payments: {'ali': tensor(5.1331), 'hassan': tensor(0.), 'mahdi': tensor(19.5805)}


In [8]:
social_welfare

tensor([50.], device='cuda:0', grad_fn=<AddBackward0>)

In [9]:
allocation

{'ali': tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0'),
 'hassan': tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0'),
 'mahdi': tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')}

[tensor([0., 0., 1., 1., 0., 1., 1., 1., 0., 0.], device='cuda:0'), tensor([1., 1., 0., 0., 0., 0., 0., 0., 1., 0.], device='cuda:0'), tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 1.], device='cuda:0')]

In [ ]:
a = torch.tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], device='cuda:0')
b = torch.tensor([0., 1., 0., 0., 0., 1., 0., 1., 0., 1.], device='cuda:0')
c = torch.tensor([1., 0., 1., 0., 1., 0., 1., 0., 1., 0.], device='cuda:0')

In [ ]:
auction.bidders[0](a) + auction.bidders[1](b) + auction.bidders[2](c)

In [ ]:
surplus = 0
for i in range(len(auction.bidders)):
  surplus += auction.bidders[i](allocations[auction.bidders[i].name]).item()

In [7]:
auction.allocation_config['scheme'] = 'GradientAscent'
auction.allocation_config['optimizer'] = 'comblearn.optim.GradientAscentOptimizer'
auction.allocation_config['learning-rate'] = 0.0001
auction.allocation_config['batch-size'] = 100
auction.allocation_config['eps'] = 0.001

In [8]:
allocations, social_welfare = auction.run()

INFO:root:Final allocation calculation...
/home/mohammad_hosseini/anaconda3/envs/auction/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
INFO:root:Bidder ali, loss: 272.4798278808594
INFO:root:Bidder hassan, loss: 454.2255859375
INFO:root:Bidder mahdi, loss: 531.08837890625
INFO:root:Step 0/640000
INFO:root:Output: 0.0


DSFValueFunction(
  (dsf): DSF(
    (layers): ParameterList(
        (0): Object of type: SCMM
        (1): Object of type: SCMM
      (0): SCMM(
        (linear): PosLinear()
        (activation): MiLU()
      )
      (1): SCMM(
        (linear): PosLinear()
        (activation): MiLU()
      )
    )
  )
)
Final Distribution: tensor([[1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]], device='cuda:0')


In [9]:
social_welfare

tensor(74., device='cuda:0')

In [14]:
allocations

{'ali': tensor([0., 0., 0., 0., 0., 0., 1., 1.], device='cuda:0'),
 'hassan': tensor([1., 0., 1., 1., 1., 0., 0., 0.], device='cuda:0'),
 'mahdi': tensor([0., 1., 0., 0., 0., 1., 0., 0.], device='cuda:0')}

In [ ]:
allocations['ali'] = torch.tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
allocations['hassan'] = torch.tensor([1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0.], device='cuda:0')
allocations['mahdi'] = torch.tensor([0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1.], device='cuda:0')

In [ ]:
surplus = 0
for i in range(len(auction.bidders)):
  surplus += auction.bidders[i](allocations[auction.bidders[i].name]).item()

In [ ]:
surplus

In [15]:
auction.bidders[0](torch.tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0'))

tensor([50.], device='cuda:0', requires_grad=True)

In [16]:
auction.bidders[0](torch.tensor([0., 0., 0., 0., 0., 0., 1., 0.,], device='cuda:0')) + auction.bidders[0](torch.tensor([1., 1., 1., 1., 1., 1., 0., 1.], device='cuda:0'))

tensor([76.], device='cuda:0', grad_fn=<AddBackward0>)

In [17]:
auction.bidders[0](torch.tensor([0, 0, 0., 0., 0., 1., 0., 0.], device='cuda:0'))

tensor([37.], device='cuda:0', requires_grad=True)

In [ ]:
import numpy as np

In [ ]:
class Branch_and_Bound():
    def __init__(self, dsfs, num_itmes, num_bidders):
        self.ws = dsfs
        self.m = num_itmes
        self.n = num_bidders
        self.best = -1000
        self.device = 'cuda:0'

    def run(self, variable_items_indexes=[], fixed_items={}):
        if len(fixed_items) == 0:
            variable_items_indexes = list(range(self.m))
        temp = self.gradient_ascent(fixed_items, variable_items_indexes)
        if temp > self.best:
            self.best = temp
        index = variable_items_indexes.copy()
        for item in index:
            variable_items_indexes.remove(item)
            for bidder in range(self.n):
                fixed_items[item] = bidder
                overestimate = self.gradient_ascent(fixed_items, variable_items_indexes)
                if overestimate <= self.best:
                    del(fixed_items[item])
                else:
                    self.run(variable_items_indexes, fixed_items)
            
            variable_items_indexes.append(item)
            variable_items_indexes.sort()
        return self.best



    
    def gradient_ascent(self, fixed_items, variable_items_indexes):
        temp = np.zeros((self.m, self.n))
        temp = self.projection_simplex_sort_2d(temp.T, fixed_items).T
        y = torch.from_numpy(temp).float().to('cuda:0')
        return self.optimize(variable_items_indexes, fixed_items, y)


    def optimize(self, variable_items_indexes, fixed_items, y, lr=2e-3, bs=10, num_iterations=100):
        y.requires_grad = True
        for i in range(num_iterations):
            s = torch.zeros(1).float().to(self.device)
            for b in range(self.n):
                s += self.ws[b](y[:,b])
            y.retain_grad()
            s.backward()
            with torch.no_grad():
                temp = y
                temp[variable_items_indexes, :] = y[variable_items_indexes, :] + lr*y.grad[variable_items_indexes, :]
                temp = temp.detach().cpu().numpy()
                temp = (self.projection_simplex_sort_2d(temp.T, fixed_items)).T
                y = None
                y = torch.from_numpy(temp).float().to('cuda:0')
            y.requires_grad = True
            for b in range(self.n):
                self.ws[b].zero_grad()
        max = 0
        for _ in range(10):
            allocation = self.generate_allocation(y)
            s = 0
            for b in range(self.n):
                s += self.ws[b](allocation[b]).item()
            if s > max:
                max = s
        for b in range(self.n):
            self.ws[b].zero_grad()
        return max

    def generate_allocation(self, y):
        output = torch.tensor([torch.multinomial(y[j], 1) for j in range(self.m)]).to(self.device)
        return [(output == i).float().to(self.device) for i in range(self.n)]

    def projection_simplex_sort_2d(self, v, fixed_items, z=1):
        """v array of shape (n_features, n_samples)."""
        p, n = v.shape
        u = np.sort(v, axis=0)[::-1, ...]
        pi = np.cumsum(u, axis=0) - z
        ind = (np.arange(p) + 1).reshape(-1, 1)
        mask = (u - pi / ind) > 0
        rho = p - 1 - np.argmax(mask[::-1, ...], axis=0)
        theta = pi[tuple([rho, np.arange(n)])] / (rho + 1)
        w = np.maximum(v - theta, 0)

        for k, v in fixed_items.items():
            w[:, k] = 0
            w[v, k] = 1
        return w

In [ ]:
dsfs = []
for bidder in auction.bidders:
    dsfs.append(bidder.vf)

In [ ]:
optimal = Branch_and_Bound(dsfs, 12, 3)

In [ ]:
socialwelfare = optimal.run()
socialwelfare

## PYSATS